<a href="https://colab.research.google.com/github/JingQian87/AppliedDL/blob/master/Lecs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading要求：

## L1: 
* DLwP: C1&2
* DL: C1
* Python DS Handbook

## L2: 
* DLwP: C3
* DL: 4.1,4.2,4.3,5.1,5.2

## L3: 
* DLwP: C4&5
* DL: 9

## L4: 
* DL: 4.1, 4.3

## L5: project相关

## L6: 
* Deep Learning: 7.12 (Dropout), 8.3 (Basic Algorithms); 8.4 (Parameter Initialization Strategies); 8.5 (Algorithms with Adaptive Learning Rates)

## L7:  C7

## L1: 
* DLwP: C1&2

# DLwP C1
## 1.1
**Machine learning**: find useful representations of input data. 各种变形，比如linear projections, translations, nonlinear operations, etc.

** Deep Learning**, 又名layered representations learning, hierarchical representations learning.
* a multistage way to learn data representations
* depth = \# layers in the model
* 每一层的weights决定了transformation 如何参数化。我们需要找到合适的weights使得input map correctly to targets.
* 衡量方式：loss function/ objective function, compute distance between prediction & true value. 得到*loss score*.
* Optimizer: 通过backpropogation 去调整weights, 从而降低loss score.






## 1.2. Brief history of ML
1. Probabilistic Modeling. 
Eg. Naive Bayes, Logistic regression.

2. Kernel Methods.
Eg: SVM

3. Decision Tree. Random Forest. Gradient Boosting Machines-> 最好的适合nonperceptual data的算法

DL优点：不用烦feature engineering.







# DLwP C2

## 2.1. MNIST example


### Step 1. Load data

In [1]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


In [2]:
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)


(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


train set 有60000 samples, test set 有10000 samples. 
每个图像为 28x28 的 NumPy 数组，像素值介于 0 到 255 之间。标签是整数数组，介于 0 到 9 之间。


### Step 2. Network

In [4]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28*28,)))
network.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


**Layer**: Data processing module, 也可视为filter for data. <br>
Layer可以提取输入的data中的representation。
通常deep learning都是由一堆简单的layer堆叠而成，成为data distillation.

在上面的命令中，network由两层Dense layer组成, 是densely connected (又称为fully connected) neural layers。
第二层是10-way softmax layer, 会输出10个probability scores (sum=1)。每个score表示图像是属于对应的digit的概率。


### Step 3. Compilation 
* **Loss Function**: 衡量network在training data上的performance。并可以据此steer itself in the right direction
* **Optimizer**: 据此network update itself
* **Metrics to monitor during training and testing**: 这里我们只关心accuracy (fraction = #correctly classified/#all).



In [0]:
network.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

### Step 4. Preparing the image data
我们知道，输入的training images，shape是(60000, 28, 28)。每个像素值在[0, 255]之间，类型是uint8。但我们想把它们变成[0, 1]之间的float 32。所以：

In [0]:
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype('float32')/255
test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype('float32')/255

<font color='red'>其实在老师给的例子里也没有这么麻烦，直接train_images=train_images/255.0，也没有把labels变成categorical。但这里不行，可能是因为那里直接从tf.keras调用？看看后面再说</font>

In [0]:
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

### Step 5. Fit data to model

In [10]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 6s 98us/step - loss: 0.2530 - acc: 0.9269
Epoch 2/5
60000/60000 [==============================] - 5s 86us/step - loss: 0.1024 - acc: 0.9694
Epoch 3/5
60000/60000 [==============================] - 5s 86us/step - loss: 0.0676 - acc: 0.9797
Epoch 4/5
60000/60000 [==============================] - 5s 86us/step - loss: 0.0492 - acc: 0.9855
Epoch 5/5
60000/60000 [==============================] - 5s 88us/step - loss: 0.0377 - acc: 0.9882


上面给出了training中的两个数值： **loss** & **accuracy**.

### Step 6. Test fitted model

In [12]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print(test_loss, test_acc)

10000/10000 [==============================] - 1s 53us/step
0.06998087865991984 0.9781


## 2.2. Data representations for neural networks

1. 讲了tensor的定义，ndim等等。

2. batch。把数据分成一部分一部分的去train. 按第一个轴（对应sample 编号）划分，所以first axis (axis 0) is called the batch axis or batch dimension

3. 常见的data tensors:
> * Vector data: 2D tensors (samples, features)
> * Timeseries data (sequence data): 3D tensors (samples, timesteps, features)
> * Images: 4D tensors (samples, height, width, channels)或者(samples, channels, height, width)
> * Video: 5D tensors (samples, frames, height, width, channels) 或者(samples, frames, channels, height, width)









## 2.3. Tensor operations

1. element-wise operation

2. Broadcasting. 不同shape的tensors间的操作，输出的ndim是输入中ndim更大的那个。<br>
 Broadcasting consists of two steps: <br>
1) Axes (called broadcast axes) are added to the smaller tensor to match the ndim of the larger tensor.<br>
2) The smaller tensor is repeated alongside these new axes to match the full shape of the larger tensor.

3. Tensor dot, 也称为tensor product. 就是点乘，比如矢量间点乘为标量；矩阵点乘矢量为矢量。

4. Tensor reshaping


## 2.4. Gradient-based Optimization